<div align="right">
  <img src="https://drive.google.com/uc?export=view&id=1J8JpP65HsHXdpJvhb_sMwn3yROyU832m" height="80" width="200" style="float: right;">
</div>
<h1><b>Data Science and Machine Learning</b></h1>
<h2><b>Clase 19</b>: Regresión lineal con regularización</h2>
<h3><b>Docente</b>: <a href="https://www.linkedin.com/in/danielablanco/">Daniela Blanco</a>

# Contenido

- [1. Regresión lineal](#lineal)
- [2. Regularización](#regularizacion)
  - [2.1. Lasso o L1](#lasso)
  - [2.2. Ridge o L2](#ridge)
- [3. Ejemplo práctico](#ejemplo)
- [4. Links de interés](#links)


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, r2_score

## 1. Regresión lineal <a name="lineal"></a>

<img src="https://drive.google.com/uc?export=view&id=19fbPfEO1uDUKc6Qr738X7LGnpEBPOpVJ" height="232" width="446" style="float: center;">

La regresión lineal (linear regression) es un tipo de modelo que se utiliza para predecir el valor de una variable dependiente (o variable objetivo) basado en el valor de una o más variables independientes (o variables predictoras).

Es decir para problemas **supervisados** de **regresión** ya que predice un valor continuo.

Si la relación entre la variable objetivo es con una única variable predictora, se dice que la **regresión** es **simple**. Si es con varias predictoras, recibe el nombre de **regresión múltiple**.

## 2. Regularización <a name="regularizacion"></a>

El sobreajuste ocurre cuando un modelo se ajusta demasiado bien a los datos de entrenamiento, capturando ruido y patrones irrelevantes, lo que resulta en un desempeño pobre en datos no vistos.

La regularización en modelos lineales es una técnica utilizada para prevenir el sobreajuste (overfitting) y mejorar la capacidad de generalización del modelo.

Agrega una penalización a la función de costo del modelo para evitar este problema.

Ambas técnicas intentan limitar o "penalizar" el tamaño de los coeficientes del modelo.

**Ventajas de la Regularización**

- Prevención del Sobreajuste: reduce la varianza del modelo y lo hace más robusto frente a nuevos datos.

- Mejora de la Interpretabilidad: Especialmente con L1, se pueden obtener modelos más simples y fáciles de interpretar, ya que puede reducir el número de características.

- Reducción de la Complejidad del Modelo: La penalización ayuda a controlar la magnitud de los coeficientes dejando algunos sin utilizar.

### 2.1. Lasso o L1 <a name="lasso"></a>

Añade una penalización igual al valor absoluto de la magnitud de los coeficientes.

Puede resultar en coeficientes iguales a cero, lo que indica que la característica correspondiente no se utiliza en el modelo.

Lo que resulta en modelos más sencillos y con menos características, también conocido como técnica de selección de características.

$$
\underset{\beta}{\text{Minimizar}} \quad \text{RSS} + \lambda \sum_{j=1}^{p} |\beta_j|
$$

Donde:
- $\text{RSS}$ es la suma de los errores cuadrados residuales.
- $\lambda$ es el parámetro de regularización.
- $|\beta_j|$ es el valor absoluto de los coeficientes.

### 2.2. Ridge o L2 <a name="ridge"></a>

Añade una penalización igual al cuadrado de la magnitud de los coeficientes.

Esto tiende a reducir los coeficientes, pero no los hace exactamente cero, por lo que todas las características se mantienen en el modelo.

Esto puede ser útil para mantener todas las características en el modelo, pero con una influencia reducida.

$$
\underset{\beta}{\text{Minimizar}} \quad \text{RSS} + \lambda \sum_{j=1}^{p} \beta_j^2
$$

Donde:
- $\text{RSS}$ es la suma de los errores cuadrados residuales: $\sum_{i=1}^n (y_i - \hat{y}_i)^2$
- $\lambda$ es el parámetro de regularización (controla la penalización).
- $\beta_j$ son los coeficientes del modelo.

## 3. Ejemplo práctico <a name="ejemplo"></a>

Podemos construir un modelo lineal regularizado fácilmente en Python utilizando la librería `scikit-learn` y las funciones `Lasso` y `Ridge`.

Hiperparámetros más importantes:

- `alpha`: Este es el hiperparámetro de regularización. Controla cuánto queremos penalizar los coeficientes altos.

Un valor más alto aumenta la regularización y, por lo tanto, los coeficientes del modelo tienden a ser más pequeños.

Por el contrario, un valor más bajo la reduce y permite coeficientes más altos.

El valor por defecto es 1.0 y su rango de valores va desde 0.0 hasta infinito.

- `max_iter`: Es el número máximo de iteraciones del modelo.

Para ejemplificar la implementación de un modelo de regresión múltiple simple utilizaremos un conjunto de datos que ya ha sido previamente tratado con un EDA completo.

## 1. Lectura del conjunto de datos procesado

In [3]:
# conexion drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
archivo = '/content/drive/MyDrive/4Geeks/cursadas/ds_pt_8/data/clean_weight-height_train.csv'
train_data = pd.read_csv(archivo)

train_data.head()

,Gender,Height,Weight
0,-1.0,-0.575639,151.275533
1,-1.0,-0.992843,123.965162
2,-1.0,-0.925964,124.765438
3,-1.0,-1.478210,119.195698
4,1.0,-1.598649,146.956646


In [5]:
train_data.shape

(8000, 3)

In [6]:
archivo = '/content/drive/MyDrive/4Geeks/cursadas/ds_pt_8/data/clean_weight-height_test.csv'
test_data = pd.read_csv(archivo)

test_data.shape

(2000, 3)

Para este problema, queremos calcular el peso (`weight`) en función de la altura (`height`) y del género (`gender`) de la persona.

Por lo tanto, el peso será la variable dependiente (variable objetivo) y la altura y el género, las variables independientes (variables predictoras).

## 2. Selección de variables

In [7]:
# seleccion de variables
X_train = train_data.drop(["Weight"], axis = 1)
y_train = train_data["Weight"]

X_test = test_data.drop(["Weight"], axis = 1)
y_test = test_data["Weight"]

## Modelado

### Regresión lineal

In [8]:
# modelo
reg_lin = LinearRegression()

# entrenamiento
reg_lin.fit(X_train, y_train)

# predicción
y_pred = reg_lin.predict(X_test)
y_pred

# metricas
mse_rl = mean_squared_error(y_test, y_pred)
rmse_rl = np.sqrt(mse_rl)
r2_rl = r2_score(y_test, y_pred)

print("MSE: ", mse_rl)
print("RMSE: ", rmse_rl)
print("Coeficiente de determinación: ", r2_rl)

MSE:  98.21235363443171
RMSE:  9.910214610916945
Coeficiente de determinación:  0.9075866115171992


Si la variable está medida en KG un RMSE de 9.91 dice que en promedio se equivoca en ese valor.

Si nos apoyamos en el valor del $R^2$, observamos que es de un 90%, un valor muy alto, luego el 90% de los datos están explicados por el modelo, por lo que es satisfactorio.

### Regularización Lasso

In [12]:
# modelo
reg_lasso = Lasso(alpha = 0.3, max_iter = 1000)

# entrenamiento
reg_lasso.fit(X_train, y_train)

# prediccion
y_pred = reg_lasso.predict(X_test)
y_pred

# metricas
mse_l1 = mean_squared_error(y_test, y_pred)
rmse_l1 = np.sqrt(mse_l1)
r2_l1 = r2_score(y_test, y_pred)

print("MSE: ", mse_l1)
print("RMSE: ", rmse_l1)
print("Coeficiente de determinación: ", r2_l1)

MSE:  98.76962192908904
RMSE:  9.938290694535405
Coeficiente de determinación:  0.9070622472239351


### Regularización Ridge

In [15]:
# modelo
reg_ridge = Ridge(alpha = 0.2, max_iter = 4000)

# entrenamiento
reg_ridge.fit(X_train, y_train)

# prediccion
y_pred = reg_ridge.predict(X_test)
y_pred

# metricas
mse_l2 = mean_squared_error(y_test, y_pred)
rmse_l2 = np.sqrt(mse_l2)
r2_l2 = r2_score(y_test, y_pred)

print("MSE: ", mse_l2)
print("RMSE: ", rmse_l2)
print("Coeficiente de determinación: ", r2_l2)

MSE:  98.21308779600086
RMSE:  9.910251651497093
Coeficiente de determinación:  0.907585920704325


### Resultados

In [16]:
# Datos
data = {
    'Modelo': ['Regresión lineal', "Regulación Lasso", "Regulación Ridge"],
    'MSE': [mse_rl, mse_l1, mse_l2],
    'RMSE': [rmse_rl, rmse_l1, rmse_l2],
    'R²': [r2_rl, r2_l1, r2_l2]
}

# Crear el DataFrame
resultados = pd.DataFrame(data)

# Mostrar el DataFrame
print(resultados)

             Modelo        MSE      RMSE        R²
0  Regresión lineal  98.212354  9.910215  0.907587
1  Regulación Lasso  98.769622  9.938291  0.907062
2  Regulación Ridge  98.213088  9.910252  0.907586


## 4. Links de interés <a name="links"></a>

- [Regularization in Machine Learning](https://towardsdatascience.com/regularization-in-machine-learning-76441ddcf99a)
- [Regularizacón con SkLearn](https://scikit-learn.org/stable/modules/linear_model.html#regularization)
- [Regularización para lograr simplicidad ](https://developers.google.com/machine-learning/crash-course/regularization-for-simplicity/video-lecture)